# Anchor bAsset Usage
> "Borrowing & depositing on the Anchor Protocol"

- toc:true
- branch: master
- badges: true
- comments: false
- author: Scott Simpson
- categories: [terra, anchor]

In [ ]:
#hide
#Imports & settings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
sns.set_theme(style="ticks", color_codes=True)
%matplotlib inline
#%load_ext google.colab.data_table
%load_ext rpy2.ipython
%R options(tidyverse.quiet = TRUE)
%R options(lubridate.quiet = TRUE)
%R options(jsonlite.quiet = TRUE)
%R suppressMessages(library(tidyverse))
%R suppressMessages(library(lubridate))
%R suppressMessages(library(jsonlite))
%R suppressMessages(options(dplyr.summarise.inform = FALSE))

dplyr.summarise.inform,[RTYPES.NILSXP]


In [ ]:
#hide
%%R
#Grab base query from Flipside
df = fromJSON('https://api.flipsidecrypto.com/api/v2/queries/1d474896-d69b-41f3-9f6f-e9c1f5a70a5f/data/latest', simplifyDataFrame = TRUE)


#Change the date to date format
df$DATE <- parse_datetime(df$DATE)

#Rename & reorder columns
df <- df %>% rename(beth_position = BETH_POSITION,
                    bluna_position = BLUNA_POSITION,
                    borrows = BORROWS,
                    collateral_position = COLLATERAL_POSITION,
                    date = DATE,
                    deposit_amount = DEPOSIT_AMOUNT,
                    eth_usd = ETH_USD,
                    luna_usd = LUNA_USD,
                    net_borrow_position = NET_BORROW_POSITION,
                    sender = SENDER,
                    ) %>%
  select(date, sender, eth_usd, luna_usd, beth_position, 
         bluna_position, collateral_position, borrows, 
         net_borrow_position, deposit_amount)

#remove the accounts with a negative loan balance - works around some errors in the source data
df_filtered<- df %>% 
  mutate(neg_borrow = if_else(net_borrow_position < 1, 1, 0)) %>%
  group_by(sender) %>%
  filter(neg_borrow == 0) %>%
  ungroup()

#loan distro - maximum loan size by wallet histogram
max_size_histo <- df_filtered %>%
  group_by(sender) %>%
  summarise(max_loan = max(net_borrow_position)) %>% arrange(desc(max_loan)) %>%
  mutate(sort_bucket = case_when(
    max_loan < 1 ~ "0-1",
    max_loan < 10 ~ "1-10",
    max_loan < 1e2 ~ "10-100",
    max_loan < 1e3 ~ "100-1k",
    max_loan < 1e4 ~ "1k-10k",
    max_loan < 1e5 ~ "10k-100k",
    max_loan < 1e6 ~ "100k-1m",
    max_loan < 1e7 ~ "1m-10m",
    max_loan < 1e8 ~ "10m-100m"
    )) %>%
  count(sort_bucket) 

#allows for ordering in the graph properly
max_size_histo$sort_bucket <- parse_factor(max_size_histo$sort_bucket,
                                            levels=c("0-1", "1-10", "10-100", "100-1k", "1k-10k", 
                                                     "10k-100k", "100k-1m", "1m-10m", "10m-100m"),
                                            ordered = TRUE)

#LTV 
df_filtered <- df_filtered %>%
  mutate(ltv = net_borrow_position / collateral_position * 100,
         collateral_type = case_when(beth_position > 0 && bluna_position > 0 ~ "bLuna+bEth",
                   bluna_position > 0 ~ "bLuna",
                   beth_position > 0 ~ "bEth")
  ) %>%
  filter(ltv != Inf) %>%  #remove the infinites due to daily rollup
  filter(ltv <= 60) %>%  #clip at max LVR due to daily rollup
  mutate(loan_size = case_when(
    net_borrow_position < 1 ~ "0-1",
    net_borrow_position < 10 ~ "1-10",
    net_borrow_position < 1e2 ~ "10-100",
    net_borrow_position < 1e3 ~ "100-1k",
    net_borrow_position < 1e4 ~ "1k-10k",
    net_borrow_position < 1e5 ~ "10k-100k",
    net_borrow_position < 1e6 ~ "100k-1m",
    net_borrow_position < 1e7 ~ "1m-10m",
    net_borrow_position < 1e8 ~ "10m-100m"
    ))

#allows for ordering in the graph properly
df_filtered$loan_size <- parse_factor(df_filtered$loan_size,
                                            levels=c("0-1", "1-10", "10-100", "100-1k", "1k-10k", 
                                                     "10k-100k", "100k-1m", "1m-10m", "10m-100m"),
                                            ordered = TRUE)
#Add a week field
df_filtered <- df_filtered %>%
  mutate(week = floor_date(date, unit="week"))

#Calculate beth/bluna collateral values split
df_collateral_split <- df_filtered %>% 
  mutate(beth_collateral = beth_position * eth_usd,
         bluna_collateral = bluna_position * luna_usd) %>%
  group_by(date) %>% 
  summarise(
    bEth = sum(beth_collateral) / (sum(beth_collateral)+sum(bluna_collateral)) * 100,
    bLuna = sum(bluna_collateral) / (sum(beth_collateral)+sum(bluna_collateral)) * 100
    ) %>%
  pivot_longer(!date, names_to="Collateral", values_to="percentage")
         



Here we attempt to answer the questions:

*What is the number of users who are using Anchor to both borrow against their bAssets and place those borrows directly in Earn? What is the average leverage LTV risk they are taking in doing this?*

Anchor is a borrow/lend protocol on the Terra Blockchain.  It accepts assets for collateral (Luna and Eth), however these assets must be bonded first - a locked form of staking.  Hence the collateral tokens accepted are bLuna and bEth.  Once these bTokens are deposited in Anchor, users are able to borrow UST against, up to a maximum Loan-to-Value ratio of 60%.  The LTV is simply the borrowed amount in UST divided by the current USD value of the collateral.  If the LTV goes above 60% due to the collateral value falling, the loan can be liquidated and the borrower will lose a portion of the capital to repay the loan.

Anchor also has a deposit facility, where users can deposit UST and earn an interest rate from it.  At the time of writing this article, the yields are very health.  Anchor deposits earn 19.44% APR.  This yield is slightly lower than the borrow interest rate - 21.1%, however there are ANC incentive rewards on the borrow side.  This gives a net borrow interest rate of 18.8% in favour of the borrower.  Yes, you read that correctly - *you are paid to borrow money and paid again to deposit back into the protocol.*






# Number of Users Borrowing & Depositing

With this deal on offer (possibly the best stablecoin yield in DeFi) it is no surprise that lots of users are taking up the offer.  In pulling these data together, there were well over 0.5m transactions on the protocol.  It was clear that users do multiple transactions - the table below shows 15,523 unique wallets who had both borrowed money from Anchor **and** deposited a sum back into the Earn portion.

*A quick note on the data aggregation method* - because of the large numbers of transactions, the data was simplified according the following rules:
- Data was aggregated by day
- A users collateral deposit & withdrawal events were summed over the day, with the resulting collateral amount being the running total of all their collateral transactions to the end of that day
- A users borrow & repayment events were also summed over the day, with the resulting net borrowing position amount being the running total of all their borrow/repay transactions to the end of that day
- Users deposit events were summed over the day

This approach simplified the data considerably (0.5m events down to 65k events) but generated a small number of edge cases which didn't make sense after aggregation.  These were driven by intra-day events - if a user deposited collateral, borrowed against it, deposited, withdrew the deposit, repaid the loan then withdrew the collateral, this would show up as a borrow event with no loan and no collateral.  These small number of edge events (1000 or so) were filtered from the dataset.  The remaining data will still be representative of user behaviour.  There is also the underlying assumption that if a user deposited on the same day they borrowed, then they were depositing the borrowed money.

In [ ]:
#hide_input
#Use the unfiltered - counts seem OK
#%unload_ext google.colab.data_table
%R df %>% distinct(sender) %>% count(name = 'Unique Wallets')

,Unique Wallets
1,15523


# How much do People Borrow?

It's worth a look at what the typical loan sizes are for the people who are borrowing and depositing back into Anchor.  The graph below shows, for each unique wallet, what was the peak borrow position for that wallet.  In other words, of all the days that the user deposited back into Anchor Earn, what was the largest loan total they over those days.  This gives us a feel for how deep each user was prepared to borrow from Anchor.

The graph below shows this distribution, with the maximum loan position along the bottom of the graph.  It shows that most wallets borrow between 100 UST and 10,000 UST.  Given the excellent returns, it's no surprise to see some big accounts.  There are over 100 wallets who had over 1m UST outstanding, and 15 who had more than 15m UST outstanding.

In [ ]:
#hide_input
#Plot the loan amount distribution
df_p = %R max_size_histo %>% arrange(sort_bucket)
fig = px.bar(df_p
             , x = "sort_bucket"
             , y = "n"
             , labels=dict(sort_bucket="Maximum Loan Position", n="Count of Wallets")
             , title= "Distribution of a Wallet's Maximum Loan Position"
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Count of Wallets')
fig.update_xaxes(title_text='Maximum Size of Loan Position (UST)')
#fig.update_traces(line_shape='spline', line_smoothing = 0.5)
fig.show()


# What Collateral Assets are Used?

When Anchor launched the only available collateral asset was bonded Luna (bLuna), available from bonding Luna tokens to a validator via the Anchor protocol.  In August 2021, Ether became an Anchor asset.  This was done via the Lido protocol on Ethereum.  Users are able to wrap ETH or stETH (ETH already staked on Lido) into a bEth token which represents their staked ETH on the Terra blockchain.  This allows users to retain long term exposure to ETH, whilst taking advantage of the yields on offer in Anchor.  Unsurprisingly, users have taken advantage of this.  The graph below shows the % of collateral assets deposited in Anchor by our users (those who are both borrowing & depositing on Anchor).  After launch, the % of bEth as collateral quickly jumped to around 5% of the total.  Expect to see this rise as more users find out about this yield opportunity.  Anchor plans to add more assets to the potential collateral pool in the future.

In [ ]:
#hide_input
#Plot the collateral split between beth & bluna
df_p = %R df_collateral_split
fig = px.area(df_p
              , x="date"
              , y="percentage"
              , color="Collateral"
              , template="simple_white"
              , width=800, height=800/1.618
              , title= "Relative Value of Collateral Assets"
              , labels=dict(date="Date", percentage="% of Anchor Collateral")              
              )
fig.update_yaxes(title_text='% of Anchor Collateral')
fig.update_xaxes(title_text=None)
fig.update_layout(legend=dict(
    yanchor="top",
    y=0.90,
    xanchor="right",
    x=0.99
))
fig.show()

# Loan to Valuation Ratio

## Average LTV

To calculate the Loan to Valuation Ratios, on each day that a user made a deposit, we took the outstanding loan position and divided it by the collateral position on that day - using that days underlying asset prices.  The table below shows the average LTV ratio for all users - around 32%.  This is around half of the liquidation cuttof of 60%, and is lower than the Anchor recommended LTV of 45%.  It appears users are relatively conservative with their loans - no surprises given how volatile the underlying assets can be.

In [ ]:
#hide_input
%R df_filtered %>% summarise("Average LTV %" = round(mean(ltv),1), "Weighted Average LTV %" = round(sum(net_borrow_position) / sum(collateral_position) * 100,1))

,Average LTV %,Weighted Average LTV %
1,31.6,32.7


## Distribution of LTVs

The graph below shows a histogram of the LTVs for all days where a wallet deposited borrowed funds in Anchor.  Standout features include a big spike around 35% - this seems to be the most popular range.  There are also spikes in 5% increments from 20% through to 45% - it seems that people gravitate towards multiples of 5.  An evolutionary impact of having 5 fingers perhaps?  It is also clear that most users borrow at less than the Anchor recommended LTV of 45%.  There are a small numbers of thrillseekers however who deposit right up to the 60% limit.  Liquidation stats for these users would be interesting.

In [ ]:
#hide_input
#Histo for LTV
df_p = %R df_filtered
fig = px.histogram(df_p
             , x = "ltv"
             #, y = "n"
             #, labels=dict(sort_bucket="Maximum Loan Position", n="Count of Wallets")
             , title= 'Distribution of Loan to Valuation Ratio %'
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Count of Loans')
fig.update_xaxes(title_text='Loan to Valuation Ratio %')
fig.add_shape(type="line"
              , x0=40
              , y0=0
              , x1=40
              , y1=2000
              , line=dict(color="Black",width=3)
)
fig.add_trace(go.Scatter(
    x=[41],
    y=[2000],
    text=["Anchor Recommended LTV"],
    mode="text"
))
fig.update_layout(showlegend=False)
fig.update_traces(textposition="middle right", selector=dict(type='scatter'))
fig.show()

## LTV by Borrow Position Size

Next we will see if there is any correlation between the size of a users outstanding borrow position and the LTV they choose when borrowing.  The graph below shows a clear trend - lower value borrowers have a lower LTV and higher borrowers take on more liquidation risk.  In particular, the accounts with >1m UST outstanding have the highest LTVs - around 38% on average.  It's possible that these users spend more time watching the markets, perhaps understand them better and are confident to take on a higher amount of risk.

The graph below this one is the same data, but shown with a boxplot.  The same trend is evident, and we see that the >1m UST borrowers have a narrower range of LTV.

In [ ]:
#hide_input
#Average LTV by borrow position size
df_p = %R df_filtered %>% group_by(loan_size) %>% summarise(ltv = mean(ltv)) %>% arrange(loan_size)
fig = px.bar(df_p
             , x = "loan_size"
             , y = "ltv"
             , labels=dict(sort_bucket="Maximum Loan Position", n="Count of Wallets")
             , title= 'Borrower LTV by Position Size'
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Average LTV %')
fig.update_xaxes(title_text='Borrowers Net Loan Position (UST)')
#fig.update_traces(line_shape='spline', line_smoothing = 0.5)
fig.show()

In [ ]:
#hide_input
#Average LTV by borrow position size - boxplot
df_p = %R df_filtered %>% arrange(loan_size) #%>% group_by(loan_size) %>% summarise(ltv = mean(ltv)) %>% arrange(loan_size)
fig = px.box(df_p
             , x = "loan_size"
             , y = "ltv"
             , labels=dict(sort_bucket="Maximum Loan Position", n="Count of Wallets")
             , title= 'Borrower LTV by Position Size Boxplot'
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Average LTV %')
fig.update_xaxes(title_text='Borrowers Net Loan Position (UST)')
#fig.update_traces(line_shape='spline', line_smoothing = 0.5)
fig.show()

## LTV Over Time

Looking at LTV over time, we see some variation - LTV ranges between around 28% at the low point, and around 35% at a high.  Interesting datapoints include the sharp drop in LTV in mid-May 2021.  THis was the time when crypto asset prices suffered sharp and substantial drops.  It's possible that higher leveraged positions were flushed out by liquidations during these rapid price falls.  The sharp rise in LTV from June through to August coincided with Luna rising in price from USD 6 to USD 30 - this obviously gave users confidence to borrow against these rising positions & take advantage of the juicy Anchor borrow & deposit rates.

In [ ]:
#hide_input
#Change over time
df_p = %R df_filtered %>% group_by(week) %>% summarise(ltv = mean(ltv)) %>% arrange(week)
fig = px.line(df_p
             , x = "week"
             , y = "ltv"
             , labels=dict(sort_bucket="Maximum Loan Position", n="Count of Wallets")
             , title= 'Borrower LTV over Time'
             , template="simple_white", width=800, height=800/1.618
             )
fig.update_yaxes(title_text='Average LTV %')
fig.update_xaxes(title_text='Borrowers Net Loan Position (UST)')
fig.update_traces(line_shape='spline', line_smoothing = 0.5)
fig.show()

## LTV by Borrow Position Size over Time

We will repeat the chart from above, but split by our borrow size cohorts from the previous sections.  There are common trends evident in the data - the May 2021 price crash seemed to impact all users.  An interesting group is the 1m-10m UST borrow group - they seem to push the LTV consistently up towards 40%, indicating a purposeful and managed approach to farming yield on Anchor.

In [ ]:
#hide_input
#Change over time
df_p = %R df_filtered %>% group_by(loan_size, week) %>% summarise(ltv = mean(ltv)) %>% arrange(loan_size, week)
fig = px.line(df_p
             , x = "week"
             , y = "ltv"
             , facet_col = 'loan_size'
             , facet_col_wrap=4
             , labels=dict(sort_bucket="Maximum Loan Position", ltv="Loan to Value Ratio %")
             , title= 'Borrower LTV by Position Size over Time'
             , template="simple_white"
             , width=1200, height=1200/1.618
             )
#fig.update_yaxes(title_text='Average LTV %')
#fig.update_xaxes(title_text='Borrowers Net Loan Position (UST)')
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.update_traces(line_shape='spline', line_smoothing = 0.5)
fig.show()

# Conclusions

The on-chain data has shown over 15,000 unique wallets which are both borrowing from Anchor and depositing in the Earn facility at the same time.  These users, on average have an LTV of around 32%.  This LTV differs depending on the size of the loan, with larger users prepared to take on more liquidation risk.  The LTV has changed over time, seemingly in response to market pricing changes in underlying assets.  In particular, there was suggestion that rapid price falls caused liquidations, leading to lower overall LTVs.

Data sourced from [Flipside Crypto:](https://www.flipsidecrypto.com/)  
[https://api.flipsidecrypto.com/api/v2/queries/1d474896-d69b-41f3-9f6f-e9c1f5a70a5f/data/latest](https://api.flipsidecrypto.com/api/v2/queries/1d474896-d69b-41f3-9f6f-e9c1f5a70a5f/data/latest)